It is possible that a later version of SRP will, instead of hashing each word to a simple binary projection,
instead hash it on the basis of all its substrings so that orthographically similar words have similar projections. That requires some additional tricks and more efficient hashing, which are being tested here.

In [1]:
import sys
import numpy as np
sys.path.append("..")
%load_ext autoreload
%autoreload 2
import sklearn

In [2]:
from SRP import Vector_file
from SRP import SRP


In [3]:
fah = SRP(32)

In [6]:
words = [l.rstrip("\n").decode("utf-8") for l in open("/Users/bschmidt/vector_models/words.txt")]

In [7]:
hasher = SRP(320)
hasher.dtype = np.int8


In [8]:
fah.hash_all_substrings("foob")

array([-6, -2, -2, -2,  4, -6,  2,  0,  4,  0, -4, -8,  0, -2,  4,  0,  0,
       -6,  0, -4,  6,  0,  2,  2, -4, -2,  4,  2,  0,  4,  2,  0], dtype=int8)

In [9]:
limit = 10000
dicto = np.zeros((limit,320))
for i in range(limit):
    dicto[i] = hasher.hash_all_substrings(words[i])

In [13]:
print len(hasher.known_hashes)
print len(hasher.recurse_dict)

62776
62776


In [11]:
hasher.known_hashes[""]

u'principle'

In [14]:
dicto[1000]

array([  5.,  -1.,   7.,  15.,   1.,   1.,   9.,  -1.,   3.,  -5.,  -7.,
        -7.,   1.,  -5.,  -1.,  -7.,  -1.,  -3.,   5.,   9.,   1.,  -3.,
         3.,   9.,  -9., -15.,   7.,   3.,  -5.,  -3.,   5.,  11.,  11.,
        -5.,  -7.,  -3.,   5.,  -3.,  -9.,   1.,   7.,   3.,  -5.,   5.,
         5.,   1.,  -7.,  -3.,   1.,  11.,  -3.,  -1.,   5.,  -3., -11.,
        -1.,  -1.,   7.,   3.,  -1.,   9.,   3.,   7.,  -1.,  -5.,  -7.,
        13.,  -1.,  -1.,   3.,   3.,  -3.,  11.,  -7.,  13.,  11.,   7.,
         9.,  -3.,   5.,  -3., -11.,  -3.,   1.,  -3.,   1.,   1.,  -7.,
        11.,   7., -11.,   5.,   7.,  -5.,  -3.,  -3.,  -3.,  -9.,  -9.,
         1.,  -7.,  -5.,   1.,  15.,  -1.,  -5.,   5.,   3.,  -3.,  -5.,
         9.,  -9.,   1.,  -3.,  11.,  11., -13.,  -3.,  -9.,   3.,   5.,
         1.,  -3.,  -1.,   7.,  -1.,   1.,  -3.,   7.,   1.,   5., -13.,
        11.,  -1.,  -7.,  13.,   3.,  -3.,  11.,  11.,   5.,  -3.,  -1.,
        -1.,  -3.,  -7.,  -1.,  -7.,  -1.,   1.,   

In [18]:
import sklearn.metrics.pairwise
def closest_to(ix,n = 10):
    ds = sklearn.metrics.pairwise.cosine_similarity(dicto[ix].reshape(1,-1),dicto)[0]
    top = np.argpartition(-ds,n)[:n]
    print
    returnt = []
    for t in top:
        returnt.append((ds[t],words[t]))
    returnt.sort(reverse = True)
    for a,b in returnt:
        print "{:>8.03} {}".format(a,b)

In [19]:
closest_to(0)

closest_to(1)

closest_to(100)

closest_to(1000)

closest_to(10000)


     1.0 the
   0.811 them
   0.804 they
   0.771 thee
   0.743 then
   0.724 these
   0.714 theme
   0.713 there
   0.692 their
   0.686 Other

     1.0 of
   0.607 o
   0.546 off
   0.529 f
   0.493 fro
   0.437 for
   0.423 do
   0.418 good
   0.399 No
   0.394 from

     1.0 upon
   0.436 Upon
   0.426 pond
   0.365 non
   0.355 weapon
   0.327 on
   0.295 weapons
   0.289 con
   0.289 won
   0.285 respond

     1.0 principle
   0.871 principles
   0.636 principal
   0.622 Principles
   0.574 principally
   0.529 prince
   0.442 princes
   0.407 princess
   0.397 print
   0.382 Prince


IndexError: index 10000 is out of bounds for axis 0 with size 10000

In [20]:
ix = 1000
ds = sklearn.metrics.pairwise.cosine_similarity(dicto[ix].reshape(1,-1),dicto)[0]


In [21]:
np.histogram(ds)

(array([ 256, 3993, 4908,  803,   31,    3,    3,    1,    1,    1]),
 array([-0.22981407, -0.10683266,  0.01614875,  0.13913015,  0.26211156,
         0.38509297,  0.50807437,  0.63105578,  0.75403719,  0.87701859,  1.        ]))

In [22]:
[word for word in words if word=="amazing" or word=="Amazing"]

[u'amazing', u'Amazing']

In [23]:
len(hasher.recurse_dict)

62776

In [21]:
out = Vector_file("out.bin",dims=320,mode="w")

In [24]:
for id,row in zip(words,dicto):
    out.add_row(id.encode("utf-8"),row.astype("<f4"))
    
out.close()

NameError: name 'out' is not defined